# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). Используйте метрику BLEU. Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [END] в текущем коде не сработает). 
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! 

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import torch
print(keras.__version__)

3.1.1


In [2]:
import numpy as np
import torch
import torch.nn as nn
import math

import keras
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [69]:
! mkdir -p data/MT
!wget -O data/MT/opus.en-ru-train.en https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget -O data/MT/opus.en-ru-train.ru https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget -O data/MT/opus.en-ru-test.en https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en
!wget -O data/MT/opus.en-ru-test.ru https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru

--2024-04-04 09:19:11--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘data/MT/opus.en-ru-train.en’

data/MT/opus.en-ru- 100%[===================>]  64.62M  37.6MB/s    in 1.7s    

2024-04-04 09:19:13 (37.6 MB/s) - ‘data/MT/opus.en-ru-train.en’ saved [67760131/67760131]

--2024-04-04 09:19:13--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘data/MT/opus.en-ru-train.ru’

data/MT/opus.en-ru- 100%[===================>] 115.72M  37.9MB/s    in 3.1s    

2024-04-04 09:19:16 (37

In [70]:
text = open('data/MT/opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('data/MT/opus.en-ru-train.ru', 'w')
f.write(text)
f.close()


In [71]:
text = open('data/MT/opus.en-ru-test.ru').read().replace('\xa0', ' ')
f = open('data/MT/opus.en-ru-test.ru', 'w')
f.write(text)
f.close()


In [72]:
en_sents = open('data/MT/opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('data/MT/opus.en-ru-train.ru').read().lower().splitlines()

In [103]:
en_sents_test = open('data/MT/opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('data/MT/opus.en-ru-test.ru').read().lower().splitlines()

In [76]:
# make uncased model

tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=[f"data/MT/opus.en-ru-{type}.en" for type in ["train", "test"]], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ], )
tokenizer_ru.train(files=[f"data/MT/opus.en-ru-{type}.ru" for type in ["train", "test"]], trainer=trainer_ru )



In [77]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [91]:
! mkdir -p tokenizers/MT
tokenizer_en.save("tokenizers/MT/tokenizer_en.json", pretty=True)
tokenizer_ru.save("tokenizers/MT/tokenizer_ru.json", pretty=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids 

In [93]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, target=True) for t in ru_sents]

In [109]:
X_en_test = [encode(t, tokenizer_en) for t in en_sents_test]
X_ru_test = [encode(t, tokenizer_ru, target=True) for t in ru_sents_test]

In [94]:
max_len_en = np.max([len(x) for x in X_en])
max_len_ru = np.max([len(x) for x in X_ru])

In [95]:
max_len_en, max_len_ru

(17863, 19379)

In [74]:
MAX_LEN_EN, MAX_LEN_RU = 33, 36

In [ ]:
PAD_ID = tokenizer_ru.token_to_id("[PAD]")
PAD_ID

In [5]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru):
        self.texts_en = [torch.LongTensor(sent)[:MAX_LEN_EN] for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_ID, )
        
        self.texts_ru = [torch.LongTensor(sent)[:MAX_LEN_RU] for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, padding_value=PAD_ID, )

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_ru = self.texts_ru[:, index]

        return ids_en, ids_ru

In [157]:
training_set = Dataset(X_en, X_ru)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True, )

valid_set = Dataset(X_en_test, X_ru_test)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=True, )

In [153]:
training_set.length

1000000

In [154]:
valid_set.length

2000

In [111]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

### Model

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: torch.Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = nn.Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: torch.Tensor,
                trg: torch.Tensor,
                src_mask: torch.Tensor,
                tgt_mask: torch.Tensor,
                src_padding_mask: torch.Tensor,
                tgt_padding_mask: torch.Tensor,
                memory_key_padding_mask: torch.Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))

        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))

        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)

        x = self.generator(outs)

        return x

    def encode(self, src: torch.Tensor, src_mask: torch.Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, tgt_mask: torch.Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory, tgt_mask)
        
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.
def generate_square_subsequent_mask(size):
    mask = (torch.triu(torch.ones((size, size), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_ID).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_ID).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask


In [326]:
from time import time
from tqdm import tqdm

def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_ru) in tqdm(enumerate(iterator)):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_ru = texts_ru.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_ru_input = texts_ru[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_ru_out = texts_ru[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in tqdm(enumerate(iterator)):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask, 
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

            
            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [174]:
torch.manual_seed(17)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_ID).to(DEVICE)

optimizer = torch.optim.AdamW(transformer.parameters(), 
                              lr=3e-4, 
                              betas=(0.9, 0.98), 
                              eps=1e-9)


In [175]:
! mkdir -p models/MT

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [331]:
from timeit import default_timer as timer
from IPython.display import clear_output

NUM_EPOCHS = 5

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)

    clear_output(wait=True)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'models/MT/model.ckpt')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'models/MT/model.ckpt')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))


Epoch: 5, Train loss: 2.640, Val loss: 2.723,            Epoch time=1952.652s


In [9]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [10]:
transformer.eval()

Seq2SeqTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerDecoderLayer(
          (self_attn): MultiheadAttent

### Translation

##### Base implementation

In [114]:
def translate(text):


    input_ids = tokenizer_en.encode(text).ids[:MAX_LEN_EN]
    output_ids = [tokenizer_ru.token_to_id('[START]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[PAD]'), tokenizer_ru.token_to_id('[END]')] and output_ids_pad.size()[0] < 150:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

        pred = logits.argmax(2)[-1].item()

    return tokenizer_ru.decode(output_ids, skip_special_tokens=True)



##### Efficient implementation

Note: we could have spawned an independent process for the needed batch length with their independent model instances (on different gpus / cpu cores) and solve the task this way -- however it seems like a massive overhead

In [59]:
from transformers import PreTrainedTokenizerFast

In [90]:
transformer = torch.load("models/MT/model.ckpt", )
tokenizer_en = PreTrainedTokenizerFast(tokenizer_file="tokenizers/MT/tokenizer_en.json")
tokenizer_ru = PreTrainedTokenizerFast(tokenizer_file="tokenizers/MT/tokenizer_ru.json")

In [91]:
en_sents_test = open('data/MT/opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('data/MT/opus.en-ru-test.ru').read().lower().splitlines()

In [92]:
tokenizer_en

PreTrainedTokenizerFast(name_or_path='', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [93]:
tokenizer_ru

PreTrainedTokenizerFast(name_or_path='', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[START]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[END]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [94]:
PAD_ID = 1

In [102]:
def encode(text, tokenizer, target=False):
    if target:
        return [2] + tokenizer.encode(text) + [3]
    else:
        return tokenizer.encode(text)

In [103]:
X_en_test = [encode(t, tokenizer_en) for t in en_sents_test]
X_ru_test = [encode(t, tokenizer_ru, target=True) for t in ru_sents_test]

In [104]:
valid_set = Dataset(X_en_test, X_ru_test)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=True, )

In [267]:
def translate_batch(text_batch):

    input_ids = [tokenizer_en.encode(text)[:MAX_LEN_EN] for text in text_batch]
    output_ids = [tokenizer_ru.encode('[START]') for text in text_batch]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_id) for input_id in input_ids]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_id) for output_id in output_ids]).to(DEVICE)
    output_ids = torch.Tensor(output_ids)

    (texts_en_mask, texts_ru_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                     texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    
    pred = logits.argmax(2).T.to("cpu")
    output_ids = torch.cat((output_ids, pred), axis = 1).long()
    
    while any([x.tolist() not in [tokenizer_ru.encode('[PAD]'), tokenizer_ru.encode('[END]')] for x in pred]) \
            and output_ids_pad.size()[0] < 150:
         
        output_ids_pad = torch.nn.utils.rnn.pad_sequence(output_ids).to(DEVICE)

        (texts_en_mask, texts_ru_mask, 
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                        texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

        pred = logits.argmax(2).T.to("cpu")[:, -1:]
        # if pred[-1] == "[END]" -> change pred to pad
        bool_mask = (output_ids[:, -1:] == 3) | (output_ids[:, -1:] == 1)
        pred[:, -1:][bool_mask] = 1
        
        output_ids = torch.cat((output_ids, pred), axis = 1).long()
    
    return [tokenizer_ru.decode(output_id, skip_special_tokens=True) for output_id in output_ids]

In [268]:
translate_batch(en_sents_test[35:40])

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


['если вы хотите увидеть это через 10 лет, вам нужно восемь до 10 лет, если только не захотите рисковать пауни становится...',
 'люксембургский форум по теме « won group », москва, 14 апреля 2008 @@ w @@ w @@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @# w @#@ w @#@ w @# w @# w @# w @# w @# w @#@ w @#@ w @# w @ w @ w @ won @ woncow @ w @ w @ w @ w @',
 'эта задача должна быть предоставлена возможность установить руководящую группу, включающей представителей контакта. @@ w @#*#@ w @#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#*#',
 'но что можно сделать по этому? @@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @#@ w @# w @ w @#@ w @#@ w @ w @ w @ w @ w @ w @ w @ w @ w @ w @ w @ w @ won @ won @ won @ won @ w @ won @ w @ w @ won @ won# won# won @ won @ won @ won @ won @ won @ won @ w',
 'когда я получу винтовки из дверимана, 

In [335]:
eng_text = "This is a target sample about a brown fox jumping over the lazy dog"

In [337]:
translate("can you translate this sentence?")

'можешь перевести это предложение?'

In [338]:
translate("sentence into russian")

'предложение в русском языке'

In [339]:
translate(eng_text)

'это мишень о коричневой лисе, подмеивший собакой собакой.'

### Evaluation

In [341]:
import nltk

In [342]:
translations = []

for text in tqdm(en_sents_test):
    translations.append(translate(text))

100%|██████████| 2000/2000 [05:23<00:00,  6.18it/s]


In [343]:
bleus = []

for i, t in tqdm(enumerate(translations)):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(round(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True), 3))

0it [00:00, ?it/s]/home/local/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/local/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/local/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

In [344]:
(sum(bleus)/len(bleus))*100

13.575249999999997

In [345]:
best_translations = np.argsort(np.array(bleus))[::-1]

In [349]:
# source text, target text, translation

list(zip(
    np.array(en_sents_test)[best_translations[:100]], 
    np.array(ru_sents_test)[best_translations[:100]], 
    np.array(translations)[best_translations[:100]]))

[('yeah.', 'да.', 'да.'),
 ('#4 長老議會 25h, wipe (00:02:05)',
  '#4 совет старейшин 25 гер., вайп (00:02:05)',
  '# 4 совет старейшин 25 гер., вайп ( 00 : 02 : 05 )'),
 ('cincinnati (ohio) 513231 **** mobile',
  'цинциннати (огайо) 513231 **** мобильный',
  'цинциннати ( огайо ) 513231 **** мобильный'),
 ('shahir...', 'шахир...', 'шахир...'),
 ('victoria (virginia) 434696 **** phone',
  'виктория (вирджиния) 434696 **** телефон',
  'виктория ( вирджиния ) 434696 **** телефон'),
 ('https://www.forumhouse.ru/forums/31/',
  'https://www.forumhouse.ru/forums/31/',
  'https :// www. forumhouse. ru / forums / 31 /'),
 ('- vivian wilkes.', '-вивиан уилкс.', '- вивиан уилкс.'),
 ('jon: what is this?', 'что это?', 'что это?'),
 ("how's the internet doing without me? it's ivan",
  'как там интернет без меня?',
  'как там интернет без меня?'),
 ('4,907usd', '4,907usd', '4, 907usd'),
 ('pepper is a gift.', 'пеппер - это подарок.', 'пеппер - это подарок.'),
 ('tina, what are you doing?',
  'тина, что

Вообще, результаты чуть-чуть выглядят как data leak из теста в трейн -- все же идеально переводятся крайне специфичные примеры. Хотя возможно это какой-то мой собственный bias


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?



Back translation -- подход, позволяющий создать параллельный корпус для обучения модели машинного перевода автоматическими методами без привлечения носителей для пополнения датасета/разметки. Тем не менее, подход предполагает наличие хотя бы ограниченно репрезентативного параллельного корпуса, позволяющего моделировать уровни языка (хотя бы семантику/морфологию и ограниченно синтаксис) для построения изначальной модели. 
Также важным условием является наличие большего по объему моноязычного корпуса на одном из языков.

Идея заключается в том, чтобы сначала обучить модель для перевода с языка, где доступен большой моноязыковой корпус, а после использовать её для расширения обучающей выборки паралельного корпуса, и последующего обучения модели обратного перевода (с языка, где большой моноязычный корпус отсутствует на один из крупных языков).

Кажется важным отметить, что по идее такой метод должен быть весьма чувствителен к чересчур однообразным параллельным текстам, а также может страдать от недостаточности размера словаря малого языка, или доменной специфичности текстов, присутствовавших в параллельном корпусе (что довольно характерно для малых языков), что скажется на обобщающей способности первоначальной модели перевода.

Итого, две модели: 
1. target-to-source,
2. source-to-target

и для запуска обучения соотвтственно; остальное -- генерация и вопросы оценки состоятельности модели, отсутсвия дрифтов в данных